# Python analysis of Project Rephetio epilepsy predictions

In [1]:
import pandas

In [2]:
pk_df = (
    pandas.read_excel('data/top-5-percent-PK-plots.xlsx', skiprows=1)
    .iloc[:100, :11]
)

pk_df['status'] = (pk_df
    [['AED', 'Anti-Epileptic properties', 'Induces seizure']]
    .astype(int).max(axis='columns')
    .map({4: 'AED', 3: 'AEP', 1: 'IS', 0: 'other'})
)

pk_df = pk_df[['Name', 'Prediction score', 'Compound Pctl', 'status']]
pk_df.head(2)

,Name,Prediction score,Compound Pctl,status
0,Topiramate,0.603,1.0,AED
1,Ethotoin,0.589,1.0,AED


In [3]:
#pk_df.rolling(window=11, min_periods=5, center=True)